In [14]:
import re
import traceback
import pandas as pd
import sqlite3
import ast
from ast import literal_eval
from Content_based import ContentRecommender
from Filter_based import FilterRecommender 

In [18]:
class CombineRecommender:
    def __init__(self, database):
        self.ct = ContentRecommender(database)
        self.fr = FilterRecommender(database)

    def recommend(self, city, roomtype, amenities, min_price, max_price, days):
        filtered_hotels = self.get_filtered_data(city, roomtype, amenities, min_price, max_price, days)
        
        if filtered_hotels.empty:
            print('No hotels available after filtering')
            return pd.DataFrame()

        listing_ids = filtered_hotels['listing_id'].tolist()
        similar_hotels = self.ct.recommend_by_amenities_topic(listing_ids)
        similiar_hotels= similar_hotels.drop_duplicates(subset='listing_id',keep='first')
        sorted_hotel= similiar_hotels.sort_values(by='review_scores_rating', ascending=False)
        return sorted_hotel
    
    def get_filtered_data(self, city, roomtype, amenities, min_price, max_price, days):
        filtered_hotels = self.fr.filter_by_all(city, roomtype, amenities, min_price, max_price, days)
        return filtered_hotels

    


In [17]:

database = 'airbnb_data.db'
hr = CombineRecommender(database)

city = "Washington DC"
roomtype = "Private room"
amenities = "Wifi"
min_price = 50
max_price = 200
days = 3

recommendations = hr.recommend(city, roomtype, amenities, min_price, max_price, days)
recommendations.head(5)


[426332090, 360451390, 401839580, 410628830, 36746760, 132477880, 386878420, 138650750, 158209830, 371795490, 73879560, 114553210, 122020790, 343031240, 203027450, 129736070, 525792000, 247558700, 411515960, 294872410, 157904490, 168117700, 406565490, 399636050, 9454260, 224166820, 129265450, 235077840, 256058610, 108609970, 187460310, 308241110, 88465420, 182374920, 159628710, 60913880, 352308240, 201263410, 118094150, 453136860, 398365310, 164654850, 159637880, 374936320, 167534380, 27910500, 60840640, 163813170, 163805620, 83530340, 372896430, 115136800, 374552360, 407485250, 179566980, 153285330, 405355740, 405619000, 8088300, 406204540, 195996460, 441199040, 441196830, 47301240, 188682000, 8284970, 348682380, 475686630, 249451140, 117143380, 411525120, 67759300, 454220870, 540166280, 3134050, 69566850, 182655370, 313908650, 507727510, 31032760, 411528870, 416462130, 64042020, 22854310, 2990300, 520563200, 48487990, 44829450, 23657260, 496600330, 1640690, 38517030, 310889800, 44120

,city,listing_id,listing_url,name,description,picture_url,latitude,longitude,room_type,bathrooms,...,beds,amenities,price,minimum_nights,maximum_nights,review_scores_rating,Dominant Ids,Percent,Topics,combined_features
2242,Washington DC,543896750,https://www.airbnb.com/rooms/54389675,1-bedroom in front of Nationals Baseball Park,This unique place has a style all its own. 1 b...,https://a0.muscache.com/pictures/a3de62a2-1432...,38.87506,-77.01218,Entire home/apt,1,...,0,"[Microwave, Hair dryer, Central heating, Bakin...",$135.00,170,365,5,"[0, 2, 1]","[0.756578, 0.100661255, 0.09467243]","[Nice, clean room, Highly recommended / Comfor...",Microwave Hair dryer Central heating Baking sh...
201,Washington DC,48412170,https://www.airbnb.com/rooms/4841217,"Fabulous1890Victorian-#1, Steps To Metro!","Don't just visit, live it! At Eastern Market M...",https://a0.muscache.com/pictures/62352416/e4a5...,38.88414,-76.99663,Entire home/apt,1,...,2,"[Hair dryer, Free street parking, Essentials, ...",$165.00,60,1125,5,"[0, 2, 3]","[0.45239177, 0.19694375, 0.18051518]","[Nice, clean room, Highly recommended / Comfor...",Hair dryer Free street parking Essentials Hot ...
1394,Washington DC,390584030,https://www.airbnb.com/rooms/39058403,? HOME SUITE HOME On The Rox ? Privacy / Parking,"** Your comfort, health, and safety are our to...",https://a0.muscache.com/pictures/98fcac78-c340...,38.96492,-77.02109,Entire home/apt,3.5,...,3,"[Self check-in, Private entrance, Carbon monox...",$204.00,31,365,5,"[2, 3, 0]","[0.3095351, 0.28817797, 0.22761907]","[Highly recommended / Comfortable, Convenient ...",Self check-in Private entrance Carbon monoxide...
2031,Washington DC,516620430,https://www.airbnb.com/rooms/51662043,"Quiet 1BR Apartment (Kitchen, W/D, Parking)",Enjoy everything DC has to offer from our rece...,https://a0.muscache.com/pictures/miso/Hosting-...,38.8976,-76.98328,Entire home/apt,1,...,1,"[Paid parking garage on premises – 1 space, Mi...",$148.00,2,30,5,"[3, 0, 2]","[0.3277092, 0.253015, 0.21868528]","[Convenient location, Nice, clean room, Highly...",Paid parking garage on premises – 1 space Micr...
551,Washington DC,162855980,https://www.airbnb.com/rooms/16285598,Lock3,"Perched on the historic C&O Canal, the Georget...",https://a0.muscache.com/pictures/miso/Hosting-...,38.90325,-77.06205,Private room,1,...,1,"[Self check-in, Carbon monoxide alarm, Fire ex...",$205.00,3,1125,5,"[1, 3, 2]","[0.4096759, 0.36322352, 0.17256665]","[Nice host, Convenient location, Highly recomm...",Self check-in Carbon monoxide alarm Fire extin...


In [9]:
minprice = 30
maxprice = 50
city = "washington dc"
roomtype = "private room"
amenities = "Wifi"
days = 14

result8=hr.fr.filter_by_all(city,roomtype,amenities,minprice,maxprice,days)
result8.head(5)

,listing_id,city,name,review_scores_rating,room_type,amenities,minimum_nights,listing_url,price
2176,536992420,washington dc,One-Room Getaway within Shared DC Urban Oasis,5,private room,"[""Microwave"", ""Dining table"", ""Free street par...",5,https://www.airbnb.com/rooms/53699242,44.0
1214,334327780,washington dc,"very quiet, clean and safe.",5,private room,"[""Hair dryer"", ""Essentials"", ""First aid kit"", ...",7,https://www.airbnb.com/rooms/33432778,50.0
1948,503333340,washington dc,Cozy room washing Dc near metro (9),4.94,private room,"[""Microwave"", ""Dining table"", ""Hair dryer"", ""F...",1,https://www.airbnb.com/rooms/50333334,44.0
393,116897500,washington dc,"Clean,Cozy, and Affordable-1",4.93,private room,"[""Microwave"", ""Dining table"", ""Free street par...",14,https://www.airbnb.com/rooms/11689750,49.0
764,201255500,washington dc,Private Room #2 15 mins to White House,4.92,private room,"[""Hair dryer"", ""Free street parking"", ""Private...",1,https://www.airbnb.com/rooms/20125550,31.0
